In [3]:
from transformers import SiglipImageProcessor, SiglipVisionModel
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
import torch
from tqdm import tqdm
import os

In [4]:
train = pd.read_parquet("train_dataset.parquet")
test = pd.read_parquet("test_dataset.parquet")

In [5]:
train

,ID,equipment,body_type,drive_type,engine_type,doors_number,color,pts,audiosistema,diski,...,fary_mult,multimedia_navigacia_mult,obogrev_mult,pamyat_nastroek_mult,podushki_bezopasnosti_mult,pomosh_pri_vozhdenii_mult,protivoygonnaya_sistema_mult,salon_mult,upravlenie_klimatom_mult,price_TARGET
0,58146,Базовая,Седан,Передний,Бензин,4,Синий,Дубликат,None,None,...,[None],[None],[None],[None],[None],[None],[None],[None],[None],51000
1,112144,Базовая,Универсал,Задний,Бензин,5,Бежевый,Оригинал,None,"14""",...,[None],[None],[None],[None],[None],[None],[Сигнализация],[None],[None],195000
2,120705,None,Внедорожник,Полный,Гибрид,5,Чёрный,Электронный,None,None,...,"[Противотуманные, Омыватели фар, Адаптивное ос...","[CD привод, MP3, Радио, TV, Экран, Управление ...","[Передних сидений, Задних сидений, Зеркал, Зад...","[Сиденья водителя, Задних сидений, Зеркал, Рул...","[Фронтальная для водителя, Коленные, Шторки, Б...","[Автопарковщик, Датчик дождя, Датчик света, Па...","[Сигнализация, Центральный замок, Иммобилайзер...","[Кожаный руль, Люк]","[Управление на руле, Атермальное остекление]",7251000
3,291392,Titanium,Седан,Передний,Бензин,4,Серебряный,Оригинал,6 колонок,"16""",...,[None],"[CD привод, MP3, Радио, TV, Экран, Управление ...","[Передних сидений, Заднего стекла]",[None],"[Фронтальная для водителя, Коленные, Шторки, Б...","[Датчик дождя, Датчик света, Парктроник задний...","[Сигнализация, Центральный замок]",[Кожаный руль],[Управление на руле],1067000
4,35742,Базовая,Седан,Передний,Бензин,4,Чёрный,Оригинал,None,None,...,[None],[None],[None],[None],[None],[None],[None],[None],[None],54000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,108955,Outdoor,Минивэн,Передний,Бензин,5,Белый,Оригинал,None,None,...,[None],[None],[None],[None],[None],[None],[None],[None],[None],883000
69996,28998,Luxe,Универсал,Передний,Бензин,5,Оранжевый,Оригинал,None,None,...,[None],[None],"[Передних сидений, Зеркал, Заднего стекла]",[None],[None],[None],[None],[None],[None],545000
69997,93671,None,Седан,Передний,Бензин,4,Золотой,Дубликат,4 колонки,"14""",...,[Противотуманные],"[MP3, Управление на руле, USB, AUX, Bluetooth]",[None],[None],[Фронтальная для водителя],[None],[Центральный замок],[None],[None],291000
69998,106131,Base,Седан,Передний,Бензин,4,Серый,Оригинал,None,None,...,[None],"[CD привод, MP3, Радио]","[Зеркал, Заднего стекла]",[None],[Фронтальная для водителя],[None],[Сигнализация],[None],[None],293000


In [6]:
processor = SiglipImageProcessor.from_pretrained("google/siglip-base-patch16-224")
model = SiglipVisionModel.from_pretrained("google/siglip-base-patch16-224")

In [7]:
def preprocess_image(image_path: str) -> torch.Tensor:
    with Image.open(image_path) as img:
        inputs = processor(images=img, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.pooler_output.squeeze(0)

def process_id(car_id: str, image_dir: str):
    images = [
        f for f in os.listdir(image_dir)
        if f.startswith(car_id)
    ]
    if not images:
        return None
    embs = [preprocess_image(os.path.join(image_dir, img)) for img in images]
    return embs[0] if len(embs) == 1 else torch.mean(torch.stack(embs), dim=0)

In [8]:
class CarIDDataset(Dataset):
    def __init__(self, df, image_dir, processor):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.processor = processor

        self.image_by_id = {}  # {id: PIL.Image}

        print("Загружаем по одному фото на каждый ID...")
        for file in tqdm(os.listdir(image_dir)):
            if not file.lower().endswith(".jpg"):
                continue
            car_id = file.split("_")[0]
            if car_id in self.image_by_id:
                continue  # уже есть фото для этого ID — пропускаем
            path = os.path.join(image_dir, file)
            try:
                img = Image.open(path).convert("RGB")
                self.image_by_id[car_id] = img
            except Exception as e:
                print(f"⚠️ Ошибка при загрузке {file}: {e}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        car_id = str(self.df.loc[idx, "ID"])
        img = self.image_by_id.get(car_id)
        if img is None:
            return None
        inputs = self.processor(images=img, return_tensors="pt")
        return inputs

In [9]:
train_dataset = CarIDDataset(train, 'train_images/', processor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False, collate_fn=lambda x: x)

Загружаем по одному фото на каждый ID...


100%|██████████| 273873/273873 [28:23<00:00, 160.80it/s] 


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

SiglipVisionModel(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(196, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attn): SiglipAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_features=3072, 

In [15]:
def get_embeddings(train, loader):
    
    hidden_dim = model.config.hidden_size
    all_embs = []

    with torch.no_grad():
        for batch in tqdm(loader):
            # batch — список dict'ов (по одному на ID), нужно обработать каждое ID отдельно
            for item in batch:
                if item is None:
                    all_embs.append(torch.zeros(hidden_dim, device=device))
                    continue
                # pixel_values: (n_images, C, H, W)
                item = {k: v.to(device) for k, v in item.items()}
                outputs = model(**item)
                emb = outputs.pooler_output.mean(dim=0)  # усредняем по фото
                all_embs.append(emb)

    return torch.stack(all_embs).cpu().numpy()

In [ ]:
train_embs_array = get_embeddings(train, train_loader)

  2%|▏         | 5/274 [00:22<19:26,  4.33s/it]